This is a cover letter builder using agentic ai


In [19]:
#imports

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
from pydantic import BaseModel

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()
    

In [8]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
name = "Sviatoslav Rutkovskyi"


In [21]:
system_prompt = f"You are a proffesional cover letter writer, and your job is to write a cover letter for {name}, highlighting {name}'s skills, experience, and achievements. \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} in the letter as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, using the tone and style suitable for a cover letter.\
Do not make up any information, and only use the information provided.\
Respond with a cover letter and nothing else.\
You will be given a job description, and you will need to tailor the cover letter to the job description."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"


In [18]:
def requestLetter(message):
    messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(requestLetter, type="messages").launch()

In [16]:



class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str
    

In [22]:
evaluator_system_prompt = f"You are an evaluator that decides whether a cover letter is acceptable. \
You are provided with {name}'s summary and LinkedIn profile, the job description, and the cover letter. \
Your task is to evaluate the cover letter, and reply with whether it is acceptable and your feedback. \
You need to check if the cover letter is professional, engaging, and tailored to the job description. \
You need to check if the cover letter was likely made by AI, and if so, you need to provide feedback on how to improve it. \
You need to check if the cover letter includes the language present in the job description. \
Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"

evaluator_system_prompt += f"With this context, please evaluate the cover letter, replying with whether the cover letter is acceptable and your feedback."

In [23]:
def evaluator_cover_letter(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt